# Multilabel Classification

## Import Library dan dataset

In [4]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import numpy as np
import pandas as pd

Membuat variabel untuk mengakses dataset

In [5]:
image_folder = "C:/Users/Marcel/Documents/Deep Learning/dataset/images/images/"
test = pd.read_csv("C:/Users/Marcel/Documents/Deep Learning/dataset/test.csv")
train = pd.read_csv("C:/Users/Marcel/Documents/Deep Learning/dataset/train.csv")
sample_submission = pd.read_csv("C:/Users/Marcel/Documents/Deep Learning/dataset/sample_submission.csv")

In [8]:
train

,class,image_path,name,xmax,xmin,ymax,ymin
0,3,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797,701,262,211
1,3,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932,786,329,238
2,3,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736,657,275,229
3,7,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986,786,136,0
4,8,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667,549,228,179
...,...,...,...,...,...,...,...
19945,4,081e7bb3832ec5bb25276db161a96274.jpg,CONSTRUCTION_ROAD,1025,600,408,148
19946,2,1ff38a7af7f13b1201d17c6e1829373a.jpg,POTHOLES,657,418,364,282
19947,2,1ff38a7af7f13b1201d17c6e1829373a.jpg,POTHOLES,507,338,436,283
19948,7,ac97490f13140fc1bfe613ec69301b34.jpg,BAD_BILLBOARD,956,713,110,25


## Eksplorasi data

In [9]:
train.name.value_counts()

GARBAGE              8597
CONSTRUCTION_ROAD    2730
POTHOLES             2625
CLUTTER_SIDEWALK     2253
BAD_BILLBOARD        1555
GRAFFITI             1124
SAND_ON_ROAD          748
UNKEPT_FACADE         127
FADED_SIGNAGE         107
BROKEN_SIGNAGE         83
BAD_STREETLIGHT         1
Name: name, dtype: int64

## Praproses data

### Sampling

In [10]:
import math
n = train.name.value_counts()
form_total = 0

for i in range(0, 11):
    form = math.ceil(n[i]/19950*2000)
    print(round(form))
    form_total = form + form_total
    if i == 10:
        print(form_total)

862
274
264
226
156
113
75
13
11
9
1
2004


In [11]:
# Define sampling variable
GARBAGE = train['name'] == "GARBAGE"
group_A = train[GARBAGE]

CONSTRUCTION_ROAD = train['name'] == "CONSTRUCTION_ROAD"
group_B = train[CONSTRUCTION_ROAD]

POTHOLES = train['name'] == "POTHOLES"
group_C = train[POTHOLES]

CLUTTER_SIDEWALK = train['name'] == "CLUTTER_SIDEWALK"
group_D = train[CLUTTER_SIDEWALK]

BAD_BILLBOARD = train['name'] == "BAD_BILLBOARD"
group_E = train[BAD_BILLBOARD]

GRAFFITI = train['name'] == "GRAFFITI"
group_F = train[GRAFFITI]

SAND_ON_ROAD = train['name'] == "SAND_ON_ROAD"
group_G = train[SAND_ON_ROAD]

UNKEPT_FACADE = train['name'] == "UNKEPT_FACADE"
group_H = train[UNKEPT_FACADE]

FADED_SIGNAGE = train['name'] == "FADED_SIGNAGE"
group_I = train[FADED_SIGNAGE]

BROKEN_SIGNAGE = train['name'] == "BROKEN_SIGNAGE"
group_J = train[BROKEN_SIGNAGE]

BAD_STREETLIGHT = train['name'] == "BAD_STREETLIGHT"
group_K = train[BAD_STREETLIGHT]

In [12]:
#Resizing the sampling
group_A_sample = group_A.sample(n=862, random_state=0)
group_B_sample = group_B.sample(n=274, random_state=0)
group_C_sample = group_C.sample(n=264, random_state=0)
group_D_sample = group_D.sample(n=226, random_state=0)
group_E_sample = group_E.sample(n=156, random_state=0)
group_F_sample = group_F.sample(n=113, random_state=0)
group_G_sample = group_G.sample(n=75, random_state=0)
group_H_sample = group_H.sample(n=13, random_state=0)
group_I_sample = group_I.sample(n=11, random_state=0)
group_J_sample = group_J.sample(n=9, random_state=0)
group_K_sample = group_K.sample(n=1, random_state=0)

In [13]:
#Merge Into dataframe
df = pd.DataFrame()
df = df.append([group_A_sample,group_B_sample,group_C_sample,group_D_sample,group_E_sample,group_F_sample,group_G_sample,group_H_sample,group_I_sample,group_J_sample,group_K_sample])
df

,class,image_path,name,xmax,xmin,ymax,ymin
13555,3,a6c589ee0340fe48bb7aec6a5e3b0c08.jpg,GARBAGE,707,659,223,198
18739,3,7e11fee29f11cd4644fd143b87e5b43f.jpg,GARBAGE,243,193,199,155
371,3,c7493cb5916f3020c26cfa29d32681b9.jpg,GARBAGE,366,264,398,353
7099,3,317a53c834291296452484fd0201c969.jpg,GARBAGE,94,-25,471,275
19886,3,1ae8bf7805901d0c53ea16d1e6007560.jpg,GARBAGE,829,686,343,233
...,...,...,...,...,...,...,...
9779,5,a3c139d7a7bee85a60d41372b24aa002.jpg,BROKEN_SIGNAGE,171,76,236,193
421,5,9b39d91113b05558b5213353a8d1331f.jpg,BROKEN_SIGNAGE,144,100,231,178
13200,5,06ba063a24fa14903cb65a3bffbc77e2.jpg,BROKEN_SIGNAGE,420,144,527,302
4879,5,aafa0f822db68e85e5c6c11fa4953b19.jpg,BROKEN_SIGNAGE,975,802,302,194


In [14]:
df.name.value_counts()

GARBAGE              862
CONSTRUCTION_ROAD    274
POTHOLES             264
CLUTTER_SIDEWALK     226
BAD_BILLBOARD        156
GRAFFITI             113
SAND_ON_ROAD          75
UNKEPT_FACADE         13
FADED_SIGNAGE         11
BROKEN_SIGNAGE         9
BAD_STREETLIGHT        1
Name: name, dtype: int64

### One-hot encoding

In [15]:
dummies = pd.get_dummies(df.name)

In [16]:
df_encode = pd.concat([df, dummies], axis='columns')
df_encode

,class,image_path,name,xmax,xmin,ymax,ymin,BAD_BILLBOARD,BAD_STREETLIGHT,BROKEN_SIGNAGE,CLUTTER_SIDEWALK,CONSTRUCTION_ROAD,FADED_SIGNAGE,GARBAGE,GRAFFITI,POTHOLES,SAND_ON_ROAD,UNKEPT_FACADE
13555,3,a6c589ee0340fe48bb7aec6a5e3b0c08.jpg,GARBAGE,707,659,223,198,0,0,0,0,0,0,1,0,0,0,0
18739,3,7e11fee29f11cd4644fd143b87e5b43f.jpg,GARBAGE,243,193,199,155,0,0,0,0,0,0,1,0,0,0,0
371,3,c7493cb5916f3020c26cfa29d32681b9.jpg,GARBAGE,366,264,398,353,0,0,0,0,0,0,1,0,0,0,0
7099,3,317a53c834291296452484fd0201c969.jpg,GARBAGE,94,-25,471,275,0,0,0,0,0,0,1,0,0,0,0
19886,3,1ae8bf7805901d0c53ea16d1e6007560.jpg,GARBAGE,829,686,343,233,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9779,5,a3c139d7a7bee85a60d41372b24aa002.jpg,BROKEN_SIGNAGE,171,76,236,193,0,0,1,0,0,0,0,0,0,0,0
421,5,9b39d91113b05558b5213353a8d1331f.jpg,BROKEN_SIGNAGE,144,100,231,178,0,0,1,0,0,0,0,0,0,0,0
13200,5,06ba063a24fa14903cb65a3bffbc77e2.jpg,BROKEN_SIGNAGE,420,144,527,302,0,0,1,0,0,0,0,0,0,0,0
4879,5,aafa0f822db68e85e5c6c11fa4953b19.jpg,BROKEN_SIGNAGE,975,802,302,194,0,0,1,0,0,0,0,0,0,0,0


In [34]:
 du = df_encode[df_encode['image_path'] == 'f0243c01844f0ab8b8173e9c8a00cc3b.jpg'] 
 for i in du['image_path']:
     print(i)
        
du

# df_encode.image_path.value_counts()

f0243c01844f0ab8b8173e9c8a00cc3b.jpg
f0243c01844f0ab8b8173e9c8a00cc3b.jpg
f0243c01844f0ab8b8173e9c8a00cc3b.jpg
f0243c01844f0ab8b8173e9c8a00cc3b.jpg


,class,image_path,name,xmax,xmin,ymax,ymin,BAD_BILLBOARD,BAD_STREETLIGHT,BROKEN_SIGNAGE,CLUTTER_SIDEWALK,CONSTRUCTION_ROAD,FADED_SIGNAGE,GARBAGE,GRAFFITI,POTHOLES,SAND_ON_ROAD,UNKEPT_FACADE
13128,3,f0243c01844f0ab8b8173e9c8a00cc3b.jpg,GARBAGE,162,97,149,111,0,0,0,0,0,0,1,0,0,0,0
13127,3,f0243c01844f0ab8b8173e9c8a00cc3b.jpg,GARBAGE,235,149,178,102,0,0,0,0,0,0,1,0,0,0,0
13132,3,f0243c01844f0ab8b8173e9c8a00cc3b.jpg,GARBAGE,107,17,161,97,0,0,0,0,0,0,1,0,0,0,0
13119,9,f0243c01844f0ab8b8173e9c8a00cc3b.jpg,CLUTTER_SIDEWALK,788,379,299,184,0,0,0,1,0,0,0,0,0,0,0


In [12]:
df_model = df_encode.drop(["class","name"], axis=1)
df_model.to_csv('df_model.csv', index=False)
df_model = pd.read_csv("C:/Users/Marcel/Documents/Deep Learning/df_model.csv")
df_model

,image_path,xmax,xmin,ymax,ymin,BAD_BILLBOARD,BAD_STREETLIGHT,BROKEN_SIGNAGE,CLUTTER_SIDEWALK,CONSTRUCTION_ROAD,FADED_SIGNAGE,GARBAGE,GRAFFITI,POTHOLES,SAND_ON_ROAD,UNKEPT_FACADE
0,a6c589ee0340fe48bb7aec6a5e3b0c08.jpg,707,659,223,198,0,0,0,0,0,0,1,0,0,0,0
1,7e11fee29f11cd4644fd143b87e5b43f.jpg,243,193,199,155,0,0,0,0,0,0,1,0,0,0,0
2,c7493cb5916f3020c26cfa29d32681b9.jpg,366,264,398,353,0,0,0,0,0,0,1,0,0,0,0
3,317a53c834291296452484fd0201c969.jpg,94,-25,471,275,0,0,0,0,0,0,1,0,0,0,0
4,1ae8bf7805901d0c53ea16d1e6007560.jpg,829,686,343,233,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999,a3c139d7a7bee85a60d41372b24aa002.jpg,171,76,236,193,0,0,1,0,0,0,0,0,0,0,0
2000,9b39d91113b05558b5213353a8d1331f.jpg,144,100,231,178,0,0,1,0,0,0,0,0,0,0,0
2001,06ba063a24fa14903cb65a3bffbc77e2.jpg,420,144,527,302,0,0,1,0,0,0,0,0,0,0,0
2002,aafa0f822db68e85e5c6c11fa4953b19.jpg,975,802,302,194,0,0,1,0,0,0,0,0,0,0,0


### Feature Extraction

In [13]:
# Menambahkan kolom area dengan perkalian dan penugasan langsung
df_model['area'] = (df_model['ymax'] - df_model['ymin']) * (df_model['xmax'] - df_model['xmin'])
model_area = df_model
model_area = model_area.drop(["xmin","xmax","ymin","ymax"], axis=1)
model_area

,image_path,BAD_BILLBOARD,BAD_STREETLIGHT,BROKEN_SIGNAGE,CLUTTER_SIDEWALK,CONSTRUCTION_ROAD,FADED_SIGNAGE,GARBAGE,GRAFFITI,POTHOLES,SAND_ON_ROAD,UNKEPT_FACADE,area
0,a6c589ee0340fe48bb7aec6a5e3b0c08.jpg,0,0,0,0,0,0,1,0,0,0,0,1200
1,7e11fee29f11cd4644fd143b87e5b43f.jpg,0,0,0,0,0,0,1,0,0,0,0,2200
2,c7493cb5916f3020c26cfa29d32681b9.jpg,0,0,0,0,0,0,1,0,0,0,0,4590
3,317a53c834291296452484fd0201c969.jpg,0,0,0,0,0,0,1,0,0,0,0,23324
4,1ae8bf7805901d0c53ea16d1e6007560.jpg,0,0,0,0,0,0,1,0,0,0,0,15730
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999,a3c139d7a7bee85a60d41372b24aa002.jpg,0,0,1,0,0,0,0,0,0,0,0,4085
2000,9b39d91113b05558b5213353a8d1331f.jpg,0,0,1,0,0,0,0,0,0,0,0,2332
2001,06ba063a24fa14903cb65a3bffbc77e2.jpg,0,0,1,0,0,0,0,0,0,0,0,62100
2002,aafa0f822db68e85e5c6c11fa4953b19.jpg,0,0,1,0,0,0,0,0,0,0,0,18684


## Import dataset kedalam array

In [14]:
SIZE = 224
import keras
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
import tensorflow.compat.v2 as tf

X_dataset = []  
for i in tqdm(range(model_area.shape[0])):
    img = image.load_img(image_folder + model_area['image_path'][i], target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255
    X_dataset.append(img)

100%|█████████████████████████████████████████████████████████████████████████████| 2004/2004 [00:16<00:00, 122.48it/s]


In [15]:
X = np.array(X_dataset)
X

array([[[[0.77254903, 0.76862746, 0.7607843 ],
         [0.77254903, 0.76862746, 0.7607843 ],
         [0.77254903, 0.76862746, 0.7607843 ],
         ...,
         [0.8039216 , 0.7882353 , 0.7764706 ],
         [0.8039216 , 0.7882353 , 0.7764706 ],
         [0.8       , 0.78431374, 0.77254903]],

        [[0.77254903, 0.76862746, 0.7607843 ],
         [0.77254903, 0.76862746, 0.7607843 ],
         [0.77254903, 0.76862746, 0.7607843 ],
         ...,
         [0.8039216 , 0.7882353 , 0.7764706 ],
         [0.8039216 , 0.7882353 , 0.7764706 ],
         [0.80784315, 0.7921569 , 0.78039217]],

        [[0.77254903, 0.76862746, 0.7607843 ],
         [0.77254903, 0.76862746, 0.7607843 ],
         [0.77254903, 0.76862746, 0.7607843 ],
         ...,
         [0.8039216 , 0.7882353 , 0.7764706 ],
         [0.8039216 , 0.7882353 , 0.7764706 ],
         [0.8039216 , 0.7882353 , 0.7764706 ]],

        ...,

        [[0.6156863 , 0.54901963, 0.47843137],
         [0.62352943, 0.57254905, 0.49803922]

In [16]:
y = np.array(model_area.drop(['image_path',"area"], axis=1))

## Memisahkan dataset menjadi 80% training set, 10% validation set dan 10% test set

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.2)
print("Training: ", len(X_train))
print("Test: ",len(X_test))

Training:  1603
Test:  401


## Transfer Learning using MobileNet

In [18]:
from keras.applications import MobileNet
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

base_model=MobileNet(weights='imagenet',include_top=False, input_shape=(SIZE, SIZE, 3)) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(11,activation='sigmoid')(x) 

model_mobileNet=Model(inputs=base_model.input,outputs=preds)

In [19]:
model_mobileNet.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model_mobileNet.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [20]:
model_mobileNet.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size = 64)

Epoch 1/10
26/26 [==============================] - 89s 3s/step - loss: 0.2392 - accuracy: 0.4772 - val_loss: 0.5719 - val_accuracy: 0.5037
Epoch 2/10
26/26 [==============================] - 86s 3s/step - loss: 0.1530 - accuracy: 0.6382 - val_loss: 0.5263 - val_accuracy: 0.5586
Epoch 3/10
26/26 [==============================] - 88s 3s/step - loss: 0.1200 - accuracy: 0.7236 - val_loss: 0.3704 - val_accuracy: 0.5187
Epoch 4/10
26/26 [==============================] - 90s 3s/step - loss: 0.1068 - accuracy: 0.7748 - val_loss: 0.5684 - val_accuracy: 0.5112
Epoch 5/10
26/26 [==============================] - 89s 3s/step - loss: 0.1028 - accuracy: 0.7910 - val_loss: 0.3204 - val_accuracy: 0.5786
Epoch 6/10
26/26 [==============================] - 87s 3s/step - loss: 0.0699 - accuracy: 0.8709 - val_loss: 0.3199 - val_accuracy: 0.5387
Epoch 7/10
26/26 [==============================] - 88s 3s/step - loss: 0.0725 - accuracy: 0.8571 - val_loss: 0.8088 - val_accuracy: 0.1746
Epoch 8/10
26/26 [==

In [21]:
from sklearn.metrics import classification_report

y_pred = model_mobileNet.predict(X_test)
y_pred = tf.argmax(y_pred, axis=1)

import numpy as np
y_true=np.argmax(y_test, axis=1)

print("Evaluation report: \n", classification_report(y_true, y_pred))

13/13 [==============================] - 4s 299ms/step
Evaluation report: 
               precision    recall  f1-score   support

           0       0.67      0.08      0.15        24
           3       0.62      0.34      0.44        47
           4       0.57      0.84      0.68        51
           5       0.00      0.00      0.00         2
           6       0.58      0.88      0.70       191
           7       0.40      0.15      0.22        13
           8       0.00      0.00      0.00        50
           9       0.00      0.00      0.00        18
          10       0.00      0.00      0.00         5

    accuracy                           0.58       401
   macro avg       0.31      0.26      0.24       401
weighted avg       0.47      0.58      0.49       401



# Transfer Learning using EfficientNet

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization

In [23]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input, decode_predictions

In [24]:
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(SIZE, SIZE, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1280, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(11, activation='sigmoid')(x)
model_efficientNet = Model(inputs=base_model.input, outputs=predictions)

model_efficientNet.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model_efficientNet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']    

 block2a_project_conv (Conv2D)  (None, 56, 56, 24)   2304        ['block2a_se_excite[0][0]']      
                                                                                                  
 block2a_project_bn (BatchNorma  (None, 56, 56, 24)  96          ['block2a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2b_expand_conv (Conv2D)   (None, 56, 56, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 56, 56, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_e

 block3b_expand_bn (BatchNormal  (None, 28, 28, 240)  960        ['block3b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block3b_expand_activation (Act  (None, 28, 28, 240)  0          ['block3b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 28, 28, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 28, 28, 240)  960        ['block3b_dwconv[0][0]']         
 )        

 D)                                                              ']                               
                                                                                                  
 block4b_bn (BatchNormalization  (None, 14, 14, 480)  1920       ['block4b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block4b_activation (Activation  (None, 14, 14, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
          

 block5a_activation (Activation  (None, 14, 14, 480)  0          ['block5a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5a_se_squeeze (GlobalAver  (None, 480)         0           ['block5a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_s

 block5c_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block5c_se_reduce[0][0]']      
                                                                                                  
 block5c_se_excite (Multiply)   (None, 14, 14, 672)  0           ['block5c_activation[0][0]',     
                                                                  'block5c_se_expand[0][0]']      
                                                                                                  
 block5c_project_conv (Conv2D)  (None, 14, 14, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 14, 14, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_d

 block6b_project_bn (BatchNorma  (None, 7, 7, 192)   768         ['block6b_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6b_drop (Dropout)         (None, 7, 7, 192)    0           ['block6b_project_bn[0][0]']     
                                                                                                  
 block6b_add (Add)              (None, 7, 7, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_e

 block6d_add (Add)              (None, 7, 7, 192)    0           ['block6d_drop[0][0]',           
                                                                  'block6c_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6d_add[0][0]']            
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 7, 7, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
          

In [25]:
model_efficientNet.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size = 64)

Epoch 1/10
26/26 [==============================] - 176s 6s/step - loss: 0.7647 - accuracy: 0.3556 - val_loss: 0.5570 - val_accuracy: 0.0449
Epoch 2/10
26/26 [==============================] - 164s 6s/step - loss: 0.4597 - accuracy: 0.5727 - val_loss: 0.3739 - val_accuracy: 0.4763
Epoch 3/10
26/26 [==============================] - 162s 6s/step - loss: 0.2723 - accuracy: 0.6631 - val_loss: 0.2816 - val_accuracy: 0.4763
Epoch 4/10
26/26 [==============================] - 164s 6s/step - loss: 0.1833 - accuracy: 0.6737 - val_loss: 0.2364 - val_accuracy: 0.4763
Epoch 5/10
26/26 [==============================] - 163s 6s/step - loss: 0.1371 - accuracy: 0.7380 - val_loss: 0.2902 - val_accuracy: 0.4763
Epoch 6/10
26/26 [==============================] - 163s 6s/step - loss: 0.0928 - accuracy: 0.8465 - val_loss: 0.3035 - val_accuracy: 0.4763
Epoch 7/10
26/26 [==============================] - 163s 6s/step - loss: 0.1133 - accuracy: 0.7891 - val_loss: 0.2761 - val_accuracy: 0.3666
Epoch 8/10
26

In [26]:
from sklearn.metrics import classification_report

y_pred = model_efficientNet.predict(X_test)
y_pred = tf.argmax(y_pred, axis=1)

import numpy as np
y_true=np.argmax(y_test, axis=1)

print("Evaluation report: \n", classification_report(y_true, y_pred))

13/13 [==============================] - 9s 631ms/step
Evaluation report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        47
           4       0.00      0.00      0.00        51
           5       0.00      0.00      0.00         2
           6       0.48      1.00      0.65       191
           7       0.00      0.00      0.00        13
           8       0.00      0.00      0.00        50
           9       0.00      0.00      0.00        18
          10       0.00      0.00      0.00         5

    accuracy                           0.48       401
   macro avg       0.05      0.11      0.07       401
weighted avg       0.23      0.48      0.31       401

